# Test Modules

In [19]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_testing_notebook, enable_autoreload, run_silent_subprocess

# Enable autoreload for development
enable_autoreload()

# Set up environment
env = setup_testing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
print_resource_summary = env['print_resource_summary']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<pre>Autoreload enabled</pre>

<pre>Project root: /scratch/edk202/word2gm-fast
TensorFlow version: 2.19.0
Device mode: GPU-enabled</pre>

<pre>Testing environment ready!</pre>

In [20]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: cm045.hpc.nyu.edu

Job Allocation:
   CPUs: 28
   Memory: 250.0 GB
   Requested partitions: short,cm,cpu_a100_2
   Actually running on: SSH failed: Host key verification failed.
   Job ID: 62891081
   Node list: cm045

GPU Information:
   Error: NVML Shared Library Not Found

TensorFlow GPU Detection:
   TensorFlow detects N/A GPU(s)
   Built with CUDA: True
============================================================</pre>

In [25]:
import datetime
from IPython.display import display, Markdown
from collections import OrderedDict
import subprocess

modules_to_test = [
    ('tests/test_corpus_to_dataset.py', 'Corpus processing'),
    ('tests/test_dataset_to_triplets.py', 'Triplet generation'),
    ('tests/test_index_vocab.py', 'Vocabulary indexing'), 
    ('tests/test_tfrecord_io.py', 'TFRecord I/O'),
    ('tests/test_word2gm_model.py', 'Word2GM model')
]

# Group tests by group label, preserving order
grouped = OrderedDict()
for test_file, group in modules_to_test:
    grouped.setdefault(group, []).append(test_file)

failures = []

result = subprocess.run([
    'pytest',
    'tests/',
    '--maxfail=0',
    '--disable-warnings',
    '-q',
    '--tb=short'
], capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print(result.stderr)

for idx, (group, test_files) in enumerate(grouped.items()):
    group_summaries = []
    for test_file in test_files:
        cmd_result = run_silent_subprocess(
            ['pytest', test_file, '--maxfail=0', '--disable-warnings', '-q', '-s'],
            capture_output=True, text=True
        )
        status = "PASSED" if cmd_result.returncode == 0 else "FAILED"
        summary_lines = []
        if cmd_result.stdout:
            for line in cmd_result.stdout.splitlines():
                line = line.strip()
                if line.startswith("[TEST"):
                    summary_lines.append(f"{status} {line}")
        if summary_lines:
            group_summaries.extend(summary_lines)
        else:
            display(Markdown(f"[{status}] No [TEST ...] summary lines found for {test_file}. Raw output:"))
            if cmd_result.stdout:
                display(Markdown(f"<pre>{cmd_result.stdout}</pre>"))
            if cmd_result.stderr:
                display(Markdown(f"<pre>{cmd_result.stderr}</pre>"))
        if cmd_result.returncode != 0:
            failures.append((group, cmd_result))
    if group_summaries:
        display(Markdown(f"<pre>{chr(10).join(group_summaries)}</pre>"))

if failures:
    print("\nDetailed output for failed tests:")
    for group, cmd_result in failures:
        print(f"\n--- {group} ---")
        if cmd_result.stdout:
            print("stdout:\n" + cmd_result.stdout)
        if cmd_result.stderr:
            print("stderr:\n" + cmd_result.stderr)

................s............                                            [100%]
28 passed, 1 skipped in 4.35s



[PASSED] No [TEST ...] summary lines found for tests/test_corpus_to_dataset.py. Raw output:

<pre>
Summary:
  retained: 15
  rejected: 17
  total: 32
[32m.[0m
[32m[32m[1m1 passed[0m[32m in 2.82s[0m[0m
</pre>

<pre>WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1751042848.049847 1878996 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751042848.054785 1878996 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751042848.068496 1878996 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042848.068513 1878996 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042848.068514 1878996 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042848.068516 1878996 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
</pre>

[PASSED] No [TEST ...] summary lines found for tests/test_dataset_to_triplets.py. Raw output:

...

<pre>Writing TFRecord to: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_write_and_load_triplets_u0/triplets.tfrecord
Write complete. Triplets written: 5
Write time: 0.01 sec
Loading TFRecord from: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_write_and_load_triplets_u0/triplets.tfrecord
TFRecord loaded and parsed
Load time (lazy initialization): 0.045 sec
[32m.[0mWriting TFRecord to: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_write_and_load_triplets_c0/triplets_compressed.tfrecord.gz
Write complete. Triplets written: 5
Write time: 0.00 sec
Loading TFRecord from: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_write_and_load_triplets_c0/triplets_compressed.tfrecord.gz
TFRecord loaded and parsed
Load time (lazy initialization): 0.012 sec
[32m.[0m[32m.[0mWriting vocabulary TFRecord to: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_write_and_load_vocab_unco0/vocab.tfrecord
Vocabulary write complete. Words written: 6
Write time: 0.00 sec
Loading vocabulary TFRecord from: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_write_and_load_vocab_unco0/vocab.tfrecord
Vocabulary loaded (optimized batched). Size: 6 words
Load time: 0.05 sec
[32m.[0mWriting vocabulary TFRecord to: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_write_and_load_vocab_comp0/vocab_compressed.tfrecord.gz
Vocabulary write complete. Words written: 6
Write time: 0.00 sec
Loading vocabulary TFRecord from: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_write_and_load_vocab_comp0/vocab_compressed.tfrecord.gz
Vocabulary loaded (optimized batched). Size: 6 words
Load time: 0.03 sec
[32m.[0m[32m.[0mSaving pipeline artifacts to: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_save_and_load_pipeline_ar0/artifacts
Writing vocabulary TFRecord to: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_save_and_load_pipeline_ar0/artifacts/vocab.tfrecord.gz
Vocabulary write complete. Words written: 6
Write time: 0.00 sec
Writing TFRecord to: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_save_and_load_pipeline_ar0/artifacts/triplets.tfrecord.gz
Write complete. Triplets written: 5
Write time: 0.00 sec
All artifacts saved successfully!
Loading pipeline artifacts from: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_save_and_load_pipeline_ar0/artifacts
Loading vocabulary TFRecord from: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_save_and_load_pipeline_ar0/artifacts/vocab.tfrecord.gz
Vocabulary loaded (optimized batched). Size: 6 words
Load time: 0.03 sec
Loading TFRecord from: /state/partition1/job-62891081/pytest-of-edk202/pytest-32/test_save_and_load_pipeline_ar0/artifacts/triplets.tfrecord.gz
TFRecord loaded and parsed
Load time (lazy initialization): 0.011 sec
All artifacts loaded successfully!
[32m.[0m
[32m[32m[1m7 passed[0m[32m in 2.87s[0m[0m
</pre>

<pre>WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1751042860.223589 1879611 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751042860.228545 1879611 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751042860.242163 1879611 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042860.242179 1879611 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042860.242180 1879611 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042860.242182 1879611 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
</pre>

[PASSED] No [TEST ...] summary lines found for tests/test_word2gm_model.py. Raw output:

<pre>[33ms[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m[32m.[0m
[32m[32m[1m12 passed[0m, [33m1 skipped[0m[32m in 2.78s[0m[0m
</pre>

<pre>WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1751042864.237896 1880116 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751042864.243282 1880116 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751042864.257796 1880116 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042864.257813 1880116 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042864.257815 1880116 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751042864.257816 1880116 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
</pre>